In [1]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXfusion.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXtexts_test.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXtokenizer_config.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3007
Total X variables: 175820

Text train example1


'35013kt 9999 prec0n CL0 CM0 10 07 q1008 27005KT 240V300 8000 SCT006 BKN009 OVC012 11/11 Q1008 TEMPO 3000 BR BKN010'


Sequences example 1


[455,
 1,
 2,
 5,
 3,
 10,
 17,
 93,
 215,
 162,
 75,
 343,
 208,
 795,
 8,
 8,
 93,
 16,
 31,
 52,
 60]


Text train example2


...

[377, 1, 2, 5, 3, 8, 8, 20, 55, 1, 238, 11, 9, 21, 4]


X


array([[  0,   0,   0,   0,   0, 455,   1,   2,   5,   3,  10,  17,  93],
       [  0,   0,   0,   0, 455,   1,   2,   5,   3,  10,  17,  93, 215],
       [  0,   0,   0, 455,   1,   2,   5,   3,  10,  17,  93, 215, 162],
       [  0,   0, 455,   1,   2,   5,   3,  10,  17,  93, 215, 162,  75],
       [  0, 455,   1,   2,   5,   3,  10,  17,  93, 215, 162,  75, 343],
       [455,   1,   2,   5,   3,  10,  17,  93, 215, 162,  75, 343, 208],
       [  1,   2,   5,   3,  10,  17,  93, 215, 162,  75, 343, 208, 795],
       [  2,   5,   3,  10,  17,  93, 215, 162,  75, 343, 208, 795,   8],
       [  5,   3,  10,  17,  93, 215, 162,  75, 343, 208, 795,   8,   8],
       [  3,  10,  17,  93, 215, 162,  75, 343, 208, 795,   8,   8,  93],
       [ 10,  17,  93, 215, 162,  75, 343, 208, 795,   8,   8,  93,  16],
       [ 17,  93, 215, 162,  75, 343, 208, 795,   8,   8,  93,  16,  31],
       [ 93, 215, 162,  75, 343, 208, 795,   8,   8,  93,  16,  31,  52],
       [  0,   0,   0,   0,   0, 377, 


Dictionary first 10 words


{'9999': 1,
 'prec0n': 2,
 'cm0': 3,
 'nosig': 4,
 'cl0': 5,
 'cavok': 6,
 '12': 7,
 '11': 8,
 '13': 9,
 '10': 10}

In [2]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/LEVX/LEVXmodel.keras")


Epoch 1/50
344/344 [==============================] - 27s 68ms/step - loss: 3.9756 - accuracy: 0.2120
Epoch 2/50
344/344 [==============================] - 23s 68ms/step - loss: 2.7674 - accuracy: 0.3198
Epoch 3/50
344/344 [==============================] - 24s 69ms/step - loss: 2.4562 - accuracy: 0.3625
Epoch 4/50
344/344 [==============================] - 24s 68ms/step - loss: 2.3422 - accuracy: 0.3765
Epoch 5/50
344/344 [==============================] - 23s 68ms/step - loss: 2.2724 - accuracy: 0.3873
Epoch 6/50
344/344 [==============================] - 23s 68ms/step - loss: 2.2127 - accuracy: 0.3943
Epoch 7/50
344/344 [==============================] - 24s 68ms/step - loss: 2.1613 - accuracy: 0.4042
Epoch 8/50
344/344 [==============================] - 24s 69ms/step - loss: 2.1185 - accuracy: 0.4092
Epoch 9/50
344/344 [==============================] - 24s 69ms/step - loss: 2.0715 - accuracy: 0.4167
Epoch 10/50
344/344 [==============================] - 23s 68ms/step - loss: 2.026